In [158]:
from workalendar.europe import *
import requests
from datetime import datetime, timedelta
import json
import itertools


In [160]:
calendarific_api_key = "2a7fb01b189f3d6735a2ab8d797b7a12dcba03d9"

def get_public_holidays_by_country(country_code):
    current_year = datetime.now().year
    current_year_str = [current_year - 1, current_year, current_year + 1]
    public_holidays = []

    try:
        for year in current_year_str:
            url = f"https://calendarific.com/api/v2/holidays?&api_key={calendarific_api_key}&country={country_code}&year={year}&type=national"
            res = requests.get(url)
            if (res.status_code == 200):
                public_holidays += [{'date': holiday['date']['datetime'], 'country': holiday['country']['id']} for holiday in res.json()['response']['holidays']]
            else:
                raise Exception(res.reason)
    except Exception as error:
        print(error)

    return public_holidays

In [148]:
is_bank_holiday = lambda date, public_holidays: any(datetime(**holiday['date']).date() == date.date() for holiday in public_holidays)

def is_business_day(date, holidays): 
    if date.weekday() >= 5:
        return False
    for holiday in holidays:
        if date.date() == datetime(**holiday['date']).date():
            return False
    return True

def find_business_date(date, public_holidays, days_before):
    count = 0
    while count < days_before:
        date -= timedelta(days=1)
        if is_business_day(date, public_holidays):
            count += 1

    return date

def find_common_business_date(event_cut_off_date, global_holidays):
    print(is_business_day(event_cut_off_date, global_holidays))
    while True:
        if (is_business_day(event_cut_off_date, global_holidays)):
            return event_cut_off_date
        event_cut_off_date -= timedelta(days=1)

def find_valid_cut_off_date(cut_off_date, days_before, fund_origin_country, countries_involved):
    country_holidays_map = {}
    all_countries = [fund_origin_country] + countries_involved

    for country_code in all_countries:
        country_holidays_map[country_code] = get_public_holidays_by_country(country_code)
    print(country_holidays_map)
    print([country_holidays_map[it] for it in [fund_origin_country] + countries_involved])
    countries_involved_holidays = list(itertools.chain(*[country_holidays_map[it] for it in all_countries]))

    fund_cut_off_date = find_business_date(cut_off_date, country_holidays_map[fund_origin_country], days_before)
    fund_common_cut_off_date = find_common_business_date(fund_cut_off_date, countries_involved_holidays)
    return fund_common_cut_off_date

In [161]:
test_date = datetime(2023, 12, 25)
date = find_valid_cut_off_date(
    cut_off_date=test_date,
    days_before=5,
    fund_origin_country='HK',
    countries_involved=['US']
)
print(date)

{'HK': [{'date': {'year': 2022, 'month': 1, 'day': 1}, 'country': 'hk'}, {'date': {'year': 2022, 'month': 2, 'day': 1}, 'country': 'hk'}, {'date': {'year': 2022, 'month': 2, 'day': 2}, 'country': 'hk'}, {'date': {'year': 2022, 'month': 2, 'day': 3}, 'country': 'hk'}, {'date': {'year': 2022, 'month': 4, 'day': 5}, 'country': 'hk'}, {'date': {'year': 2022, 'month': 4, 'day': 15}, 'country': 'hk'}, {'date': {'year': 2022, 'month': 4, 'day': 16}, 'country': 'hk'}, {'date': {'year': 2022, 'month': 4, 'day': 18}, 'country': 'hk'}, {'date': {'year': 2022, 'month': 5, 'day': 1}, 'country': 'hk'}, {'date': {'year': 2022, 'month': 5, 'day': 2}, 'country': 'hk'}, {'date': {'year': 2022, 'month': 5, 'day': 8}, 'country': 'hk'}, {'date': {'year': 2022, 'month': 5, 'day': 9}, 'country': 'hk'}, {'date': {'year': 2022, 'month': 6, 'day': 3}, 'country': 'hk'}, {'date': {'year': 2022, 'month': 7, 'day': 1}, 'country': 'hk'}, {'date': {'year': 2022, 'month': 9, 'day': 12}, 'country': 'hk'}, {'date': {'ye